## Twitter Sentimental Analysis with Emoji

###---- TRY 1

In [ ]:
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

- Load the dataset (example dataset: Sentiment140)
- You can download the dataset from: http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
-  Ensure you extract the zip file and place the CSV files in the appropriate path

In [ ]:
data = pd.read_csv(r'/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None,error_bad_lines=False, engine="python")

<ipython-input-9-0903634305c4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(r'/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None,error_bad_lines=False, engine="python")
Skipping line 203629: unexpected end of data


In [ ]:
# Preprocessing
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data['target'] = data['target'].replace(4, 1)  # Convert positive class to 1

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [ ]:
padded_sequences

array([[  1,  68, 240, ...,   0,   0,   0],
       [  8, 535,  21, ...,   0,   0,   0],
       [  1,   2,   1, ...,   0,   0,   0],
       ...,
       [735, 110,  24, ...,   0,   0,   0],
       [146,  36, 143, ...,   0,   0,   0],
       [ 62, 313,   5, ...,   0,   0,   0]], dtype=int32)

In [ ]:
# Define LSTM model
model = Sequential()
model.add(Embedding(10000, 128, input_length=100))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
len(X)

203628

In [ ]:
len(y)

203628

In [ ]:
# Train the model
X = padded_sequences
y = data['target'].values
model.fit(X, y, batch_size=64, epochs=3, validation_split=0.2)

Epoch 1/3
2546/2546 [==============================] - 658s 258ms/step - loss: 1.0541e-04 - accuracy: 1.0000 - val_loss: 1.9701e-05 - val_accuracy: 1.0000
Epoch 2/3
2546/2546 [==============================] - 696s 273ms/step - loss: 9.6177e-06 - accuracy: 1.0000 - val_loss: 4.1155e-06 - val_accuracy: 1.0000
Epoch 3/3
2546/2546 [==============================] - 638s 251ms/step - loss: 2.2467e-06 - accuracy: 1.0000 - val_loss: 1.0736e-06 - val_accuracy: 1.0000


In [ ]:
# Test the model
# test_text = ["Oh so good", "This is terrible 😞"]
test_text=["Oh this is very good"]
test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded_sequences = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
predictions = model.predict(test_padded_sequences)


1/1 [==============================] - 0s 33ms/step


In [ ]:
predictions

array([[1.0736179e-06]], dtype=float32)

In [ ]:
for i in range(len(test_text)):
    sentiment = "positive" if predictions[i] > 0.5 else "negative"
    print(f"Text: {test_text[i]} - Sentiment: {sentiment}")

# Note: Make sure to adjust the paths, hyperparameters, and model architecture based on your dataset and requirements.

Text: Oh this is very good - Sentiment: negative


###--- TRY 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Load and preprocess data
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None, on_bad_lines=False, engine="python")
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data.drop(['id', 'date', 'flag', 'user'], axis=1, inplace=True)
data['target'] = data['target'].map({0: 0, 4: 1})
X = data['text']
y = data['target']

# Tokenize and pad sequences
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=max_len)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build RNN model with dropout regularization
model = Sequential()
model.add(Embedding(max_words, 32))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))  # Add dropout
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)


<ipython-input-1-cbc72c4fcda8>:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None, error_bad_lines=False, engine="python")
Skipping line 49423: unexpected end of data


Epoch 1/5
248/248 [==============================] - 61s 230ms/step - loss: 0.0629 - accuracy: 0.9966 - val_loss: 5.7593e-04 - val_accuracy: 1.0000
Epoch 2/5
248/248 [==============================] - 50s 201ms/step - loss: 3.8565e-04 - accuracy: 1.0000 - val_loss: 2.4029e-04 - val_accuracy: 1.0000
Epoch 3/5
248/248 [==============================] - 48s 195ms/step - loss: 1.9110e-04 - accuracy: 1.0000 - val_loss: 1.3895e-04 - val_accuracy: 1.0000
Epoch 4/5
248/248 [==============================] - 49s 199ms/step - loss: 1.1827e-04 - accuracy: 1.0000 - val_loss: 9.2050e-05 - val_accuracy: 1.0000
Epoch 5/5
309/309 [==============================] - 6s 20ms/step - loss: 6.5752e-05 - accuracy: 1.0000
Test loss: 6.575222505489364e-05
Test accuracy: 1.0


In [ ]:
data['target'].unique()

array([0])

In [ ]:
test_text=["Oh this is very good"]
test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded_sequences = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
predictions = model.predict(test_padded_sequences)

In [ ]:
df

,0
0,"0,""1556263217"",""Sat Apr 18 22:11:36 PDT 2009"",..."


In [ ]:
len(df)

42946

In [ ]:
df=pd.read_csv(r'/content/Dataset-SA.csv',engine='python',on_bad_lines='warn')

In [ ]:
df['Sentiment'].value_counts()

positive    34988
negative     6071
neutral      1886
Name: Sentiment, dtype: int64

In [ ]:
df.columns

Index(['product_name', 'product_price', 'Rate', 'Review', 'Summary',
       'Sentiment'],
      dtype='object')

In [ ]:
df.sample(5)

,product_name,product_price,Rate,Review,Summary,Sentiment
10499,Rixen Cricket Net 20x10 Feet Cricket Net??????...,349,1,waste of money!,pakegik so bad,negative
9127,Eurodomo Hood Indigo PRO HC SC FL BK 60 Auto C...,7599,5,worth every penny,chimney is looking great and and till now its ...,positive
2199,boAt Rockerz 510 Super Extra Bass Bluetooth He...,1199,5,simply awesome,the headphones are amazing people who love bas...,positive
16990,"Pigeon Favourite Electric Kettle??????(1.5 L, ...",699,3,fair,expensive,neutral
33581,"Sound Boss Androidify 3rd Generation 9"" Inch A...",7999,3,decent product,performance is not so good lcd touch response ...,negative


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv(r'/content/Dataset-SA.csv',engine='python',on_bad_lines='warn')

# Preprocess the data
X = data['Review'].values
y = data['Sentiment'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data

# Convert elements to strings
X_train = [str(x) for x in X_train]
# Convert elements to strings
X_test = [str(x) for x in X_test]

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad sequences for input to the LSTM model
max_len = max(len(sequence) for sequence in X_train)
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = EarlyStopping(patience=2, monitor='val_loss')
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=64, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10


ValueError: ignored

In [ ]:
X_train

array(['wonderful', 'wonderful', 'slightly disappointed', ...,
       'worth every penny', 'value-for-money', 'mind-blowing purchase'],
      dtype=object)

In [ ]:
#predicted test
# Input your custom text
custom_text = "This product is amazing! I love it."

# Preprocess the custom text
custom_text = tokenizer.texts_to_sequences([custom_text])
custom_text = pad_sequences(custom_text, maxlen=max_len, padding='post')

# Predict sentiment
sentiment_labels = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_sentiment = trained_model.predict(custom_text)
predicted_label = sentiment_labels[np.argmax(predicted_sentiment)]

print("Predicted Sentiment:", predicted_label)
# In this snippet, replace 'sentiment_analysis_lstm_model.h5' with the path to your trained LSTM model file. After inputting your custom text, the code preprocesses the text, predicts the sentiment label, and displays the predicted sentiment.







In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.utils.class_weight import compute_class_weight

# Load the dataset
df = pd.read_csv('/content/Dataset-SA.csv')

# Preprocess the data
X = df['Review'].astype(str)
y = df['Sentiment']

# Convert labels to numerical values
sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
y = y.map(sentiment_mapping)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, padding='post', truncating='post')

from keras.layers import Dropout, BatchNormalization

model = Sequential()
model.add(Embedding(max_words, 32))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

from keras.optimizers import Adam

# Create an Adam optimizer with a custom learning rate
learning_rate = 0.001  # You can adjust this value as needed
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=[0, 1, 2], y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}

# Train the model
from keras.utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

model.fit(X_train_padded, y_train_encoded, epochs=3, validation_data=(X_test_padded, y_test_encoded),class_weight=class_weights_dict)



In [ ]:
# Predict sentiment with custom text
def predict_sentiment(text):
    text_sequence = tokenizer.texts_to_sequences([text])
    text_padded = pad_sequences(text_sequence, maxlen=100, padding='post', truncating='post')
    sentiment_probabilities = model.predict(text_padded)[0]
    sentiment = ['positive', 'neutral', 'negative'][sentiment_probabilities.argmax()]
    return sentiment



In [ ]:
# Test custom prediction

custom_text = input("Enter the text > ")
predicted_sentiment = predict_sentiment(custom_text)
print(f"Predicted sentiment: {predicted_sentiment}")


Enter the text > oh wow nice man
1/1 [==============================] - 0s 35ms/step
Predicted sentiment: negative


In [ ]:
predicted_sentiment

'positive'

## YT videos